In [263]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
from lxml import etree
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from time import sleep
import re

In [179]:
headers = {'User-Agent': 'Animated Winter'}

summaryTable = pd.DataFrame()

for i in range(1,36):
    r = requests.get('https://boardgamegeek.com/browse/boardgame/page/{}?sort=rank'.format(i), headers=headers)
    bs = BeautifulSoup(r.text)
    table = bs.findAll(lambda tag: tag.name=='table' and tag.has_attr('class'))
    tableHTML = table[2]
    urls = [np.where(tag.has_attr('href'),tag.get('href'),"no link") for tag in tableHTML.find_all('a')]
    urls = pd.DataFrame(urls)
    pandasTable = pd.read_html(str(tableHTML), header=0)[0]
    
    urls[0] = urls[0].astype(str)
    urls = urls[urls[0].str.contains("/boardgame/")][1:]
    urls = urls[~urls[0].str.contains("/marketplace/")]
    urls = pd.unique(urls[0])
    urls = pd.DataFrame(urls)

    pandasTable['link'] = urls
    pandasTable = pandasTable.drop(['Unnamed: 1', 'Shop'], axis = 1)
    
    if summaryTable.empty:
        summaryTable = pandasTable
    else:
        summaryTable = pd.concat([summaryTable, pandasTable],ignore_index=True)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


In [191]:
display(summaryTable)
type(summaryTable["Geek Rating"][0])

,Board Game Rank,Title,Geek Rating,Avg Rating,Num Voters,link
0,1,Gloomhaven (2017),8.611,8.91,23998,/boardgame/174430/gloomhaven
1,2,Pandemic Legacy: Season 1 (2015),8.492,8.65,30624,/boardgame/161936/pandemic-legacy-season-1
2,3,Through the Ages: A New Story of Civilization ...,8.258,8.54,15341,/boardgame/182028/through-ages-new-story-civil...
3,4,Terraforming Mars (2016),8.241,8.40,37459,/boardgame/167791/terraforming-mars
4,5,Twilight Struggle (2005),8.175,8.32,34022,/boardgame/12333/twilight-struggle
5,6,Star Wars: Rebellion (2016),8.161,8.45,16260,/boardgame/187645/star-wars-rebellion
6,7,Gaia Project (2017),8.125,8.56,8845,/boardgame/220308/gaia-project
7,8,Scythe (2016),8.121,8.29,38506,/boardgame/169786/scythe
8,9,Great Western Trail (2016),8.078,8.29,18631,/boardgame/193738/great-western-trail
9,10,Twilight Imperium (Fourth Edition) (2017),8.064,8.74,6260,/boardgame/233078/twilight-imperium-fourth-edi...


numpy.float64

In [195]:
summaryTable.to_pickle("Summary.p")

In [12]:
#########

In [231]:
driver = webdriver.Firefox(r"G:\OneDrive - Uniwersytet Warszawski\Projekty\2018.10.18 Data Science\Data Science\!Projects\animated-winter", timeout=1)

In [343]:
detailedTable = pd.DataFrame(columns=['Title','Average Rating','Number of ratings','Std.dev','Complexity','Number of Comments','Fans','Page Views','Overall Ranking','All Time Plays','Played This Month', 'Own', 'Previously Owned','For Trade','Want in Trade','Wishlist','Has Parts','Want Parts'])
detailedTable2 = pd.DataFrame()

for j in range(101):
    driver.get('https://boardgamegeek.com{}/stats'.format(summaryTable['link'][j]))
    pageSource = driver.page_source
    bs = BeautifulSoup(pageSource, "lxml")
    data = bs.findAll('div',{'class':'outline-item-description'})
    
    data2 = []
    for i in data:
        data2.append(i.get_text().strip())

    for i in range(len(data2)):
        data2[i] = data2[i].replace(",","")
        #data2[i] = data2[i].replace(".",",")

    data2[3] = data2[3][0:4]
    data2[0] = float(data2[0])
    data2[1] = int(data2[1]) 
    data2[2] = float(data2[2])
    data2[3] = float(data2[3]) 

    for i in range(len(data2[4:])+1):
        i = i+3
        data2[i] = int(data2[i])

    if len(data2) == 19:
        data2.pop(8)
        data2.pop(8)
    else:
        data2.pop(8)
    
    print(j)
    data2.insert(0,summaryTable['Title'][j]) 
    detailedTable.loc[len(detailedTable)] = data2

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [344]:
display(detailedTable)

,Title,Average Rating,Number of ratings,Std.dev,Complexity,Number of Comments,Fans,Page Views,Overall Ranking,All Time Plays,Played This Month,Own,Previously Owned,For Trade,Want in Trade,Wishlist,Has Parts,Want Parts
0,Gloomhaven (2017),8.907,23998,1.60,3,4770,4621,7984727,1,168604,822,39530,1206,199,1342,11895,17,7
1,Pandemic Legacy: Season 1 (2015),8.646,30624,1.61,2,4905,2217,2801958,2,176542,177,48726,3049,236,786,8386,8,4
2,Through the Ages: A New Story of Civilization ...,8.537,15341,1.49,4,2246,1097,2208830,3,42406,126,19073,1024,238,1034,6362,6,4
3,Terraforming Mars (2016),8.403,37459,1.36,3,5663,2930,4022919,4,178194,648,47979,1356,210,2086,12817,15,5
4,Twilight Struggle (2005),8.325,34022,1.59,3,7697,3680,4311524,5,88007,118,46741,3134,763,1341,8874,36,27
5,Star Wars: Rebellion (2016),8.447,16260,1.38,3,2635,1643,2716540,6,24193,41,24772,879,187,1347,7873,4,6
6,Gaia Project (2017),8.561,8845,1.42,4,1341,774,1303095,7,26363,147,10933,316,55,1171,6018,1,2
7,Scythe (2016),8.290,38506,1.42,3,6090,4125,4961293,8,120561,394,50113,1635,286,1862,13097,25,9
8,Great Western Trail (2016),8.286,18631,1.25,3,2535,905,1605989,9,56174,187,22742,848,115,1624,6901,7,3
9,Twilight Imperium (Fourth Edition) (2017),8.737,6260,1.56,4,1059,895,1267051,10,8313,29,8703,159,37,717,4741,0,1


In [346]:
detailedTable.to_pickle("Detailed.p")